In [0]:
-- data_ref = '2017-06-01'

with params as (
  select
    '2017-06-01' as dt_ref
),

sellers as (
  select
    idVendedor,
    descUF as vendedorDescUF,
    case when descUF in ('SP', 'RJ', 'MG', 'ES') then 'SUDESTE'
         when descUF IN ('PR', 'SC', 'RS') THEN 'SUL'
         when descUF in ('RO', 'AC', 'AM', 'RR', 'AP', 'PA', 'TO') then 'NORTE'
         when descUF in ('MT', 'MS', 'GO', 'DF') then 'CENTRO-OESTE'
         when descUF in ('BA', 'SE', 'AL', 'CE', 'MA', 'PE') then 'NORDESTE' end as vendedorRegiao
  from silver.olist.vendedor
),

orders as (
  select
    p.idPedido,
    p.idCliente,
    c.descUF as clienteDescUF,
    ip.idVendedor,
    ip.vlPreco
  from silver.olist.pedido as p
  left join silver.olist.item_pedido as ip
    on ip.idPedido = p.idPedido
  left join silver.olist.cliente as c
    on c.idCliente = p.idCliente
  where date(p.dtPedido) < (select date(dt_ref) from params)
    and idVendedor is not null
),

states as (
  select
    distinct
    clienteDescUF
  from orders
),

sellers_and_states as (
  select
    idVendedor,
    clienteDescUF
  from sellers
  cross join states
),

gmv_by_seller_state as (
  select
    ss.idVendedor,
    ss.clienteDescUF,
    coalesce(sum(o.vlPreco), 0) as GMV_by_seller_and_state
  from sellers_and_states as ss
  left join orders as o
    on o.idVendedor = ss.idVendedor
    and o.clienteDescUF = ss.clienteDescUF
  group by all
),

gmv_by_state as (
  select
    clienteDescUF,
    sum(vlPreco) as GMV_by_state
  from orders
  group by all
),

fs_seller as (
select
  (select date(dt_ref) from params) as dtRef,
  s.idVendedor,
  s.vendedorDescUF,
  s.vendedorRegiao,
  max(gs2.GMV_by_state) as gmvEstadoVendedor,
  max(case when gss.clienteDescUF = 'AC' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'AC' then gs1.GMV_by_state end) as shareGmvAC,
  max(case when gss.clienteDescUF = 'AL' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'AL' then gs1.GMV_by_state end) as shareGmvAL,
  max(case when gss.clienteDescUF = 'AM' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'AM' then gs1.GMV_by_state end) as shareGmvAM,
  max(case when gss.clienteDescUF = 'AP' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'AP' then gs1.GMV_by_state end) as shareGmvAP,
  max(case when gss.clienteDescUF = 'BA' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'BA' then gs1.GMV_by_state end) as shareGmvBA,
  max(case when gss.clienteDescUF = 'CE' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'CE' then gs1.GMV_by_state end) as shareGmvCE,
  max(case when gss.clienteDescUF = 'DF' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'DF' then gs1.GMV_by_state end) as shareGmvDF,
  max(case when gss.clienteDescUF = 'ES' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'ES' then gs1.GMV_by_state end) as shareGmvES,
  max(case when gss.clienteDescUF = 'GO' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'GO' then gs1.GMV_by_state end) as shareGmvGO,
  max(case when gss.clienteDescUF = 'MA' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'MA' then gs1.GMV_by_state end) as shareGmvMA,
  max(case when gss.clienteDescUF = 'MG' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'MG' then gs1.GMV_by_state end) as shareGmvMG,
  max(case when gss.clienteDescUF = 'MS' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'MS' then gs1.GMV_by_state end) as shareGmvMS,
  max(case when gss.clienteDescUF = 'MT' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'MT' then gs1.GMV_by_state end) as shareGmvMT,
  max(case when gss.clienteDescUF = 'PA' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'PA' then gs1.GMV_by_state end) as shareGmvPA,
  max(case when gss.clienteDescUF = 'PB' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'PB' then gs1.GMV_by_state end) as shareGmvPB,
  max(case when gss.clienteDescUF = 'PE' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'PE' then gs1.GMV_by_state end) as shareGmvPE,
  max(case when gss.clienteDescUF = 'PI' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'PI' then gs1.GMV_by_state end) as shareGmvPI,
  max(case when gss.clienteDescUF = 'PR' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'PR' then gs1.GMV_by_state end) as shareGmvPR,
  max(case when gss.clienteDescUF = 'RJ' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'RJ' then gs1.GMV_by_state end) as shareGmvRJ,
  max(case when gss.clienteDescUF = 'RN' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'RN' then gs1.GMV_by_state end) as shareGmvRN,
  max(case when gss.clienteDescUF = 'RO' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'RO' then gs1.GMV_by_state end) as shareGmvRO,
  max(case when gss.clienteDescUF = 'RR' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'RR' then gs1.GMV_by_state end) as shareGmvRR,
  max(case when gss.clienteDescUF = 'RS' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'RS' then gs1.GMV_by_state end) as shareGmvRS,
  max(case when gss.clienteDescUF = 'SC' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'SC' then gs1.GMV_by_state end) as shareGmvSC,
  max(case when gss.clienteDescUF = 'SE' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'SE' then gs1.GMV_by_state end) as shareGmvSE,
  max(case when gss.clienteDescUF = 'SP' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'SP' then gs1.GMV_by_state end) as shareGmvSP,
  max(case when gss.clienteDescUF = 'TO' then gss.GMV_by_seller_and_state end)
    / max(case when gs1.clienteDescUF = 'TO' then gs1.GMV_by_state end) as shareGmvTO
from sellers as s
left join gmv_by_seller_state as gss
  on gss.idVendedor = s.idVendedor
left join gmv_by_state as gs1
  on gs1.clienteDescUF = gss.clienteDescUF
left join gmv_by_state as gs2
  on gs2.clienteDescUF = s.vendedorDescUF
group by all
)

select *
from fs_seller